In [57]:
# store start time to get execution time of entire script
import time
start_time = time.time()

In [58]:
import numpy as np
np.random.seed(2017) # set random seed value to get reproducible results

In [59]:
from sklearn.model_selection import cross_validate
from sksurv.ensemble import RandomSurvivalForest
from tqdm.notebook import tqdm
import statistics

def n_permutations(n, rsf, feat, X, y):
    mean = 0
    std = 0
    for i in range(n):
        X[feat] = np.random.permutation(X[feat].values)        
        scores = cross_validate(rsf, X, y, cv=5)
        mean += scores['test_score'].mean()
        std += statistics.pstdev(scores['test_score'])
    #print('Feature:', feat, ' | Mean Score:', mean/n, ' | Standard Deviation:', std/n)
    return mean/n, std/n

def random_forest(X, y, label):
    # run model and cross validate to get concordance
    rsf = RandomSurvivalForest()
    
    scores = cross_validate(rsf, X, y, cv=5)
    concordance = scores['test_score'].mean()
    print('RF score:', concordance)
    
    # re-train model on full dataset
    rsf = RandomSurvivalForest()
    rsf.fit(X, y)
    
    # calculate feature importances
    feature_importance_rf = pd.DataFrame({'Feature':list(X.columns),})
    feature_importance_rf[label] = 0
        
    for i,row in tqdm(feature_importance_rf.iterrows(), total=feature_importance_rf.shape[0]):
        feat = row['Feature']
        temp_data = X.copy()
        temp_score, _ = n_permutations(5, rsf, feat, temp_data, y)
        percent_change = (concordance - temp_score) / concordance * 100 # percent change
        if percent_change < 0:
            percent_change = 0 # removing feature helped model, should not be reflected in feature importance
        feature_importance_rf.iloc[i, feature_importance_rf.columns.get_loc(label)] = percent_change
    
    # return scores and models
    return rsf, concordance, feature_importance_rf

In [60]:
def get_lasso_features(X):
    # one-hot encode all variables (except primsev) to get hazards across groups, drop reference group
    features_to_ignore = ['female','nonwhite','unemplmt_cd','primsev_alcohol','primsev_amphetamines','homeless_0_cd',
                          'primsev_cocaine','primsev_marijuana','primsev_opioids','primsev_other']
    lasso_X = X.copy()
    
    for col in lasso_X.columns:
        if col not in features_to_ignore:
            one_hot = pd.get_dummies(lasso_X[col], prefix=col)
            one_hot = one_hot.loc[:, ~one_hot.columns.str.endswith('1')] # drop group and use as reference
            lasso_X = lasso_X.drop(col,axis = 1)
            lasso_X = lasso_X.join(one_hot)
    #print('Lasso Features:',lasso_X.columns)
    return lasso_X

In [61]:
from sksurv.linear_model import CoxnetSurvivalAnalysis

def lasso_regression(X, y, label):
    # l1_ratio = 1 adjusts model to implement LASSO method for penalties
    rcr = CoxnetSurvivalAnalysis(l1_ratio=1)
    
    scores = cross_validate(rcr, X, y, cv=5)
    concordance = scores['test_score'].mean()
    print('Lasso score:', concordance)
    
    # fit_baseline_model = True allows us to create survival/hazard plots after model is fit
    rcr = CoxnetSurvivalAnalysis(fit_baseline_model=True, l1_ratio=1)
    rcr.fit(X, y)
    
    # feature importances from Lasso
    feature_importance_lasso = pd.DataFrame({'Feature':list(X.columns), 
                                                  label:np.average(rcr.coef_, weights=rcr.alphas_, axis = 1),})
    # convert regression coefficients to hazard ratios
    feature_importance_lasso[label] = np.exp(feature_importance_lasso[label])
    # rank by magnitude of deviation from 1
    feature_importance_lasso[label + '_adjusted'] = np.absolute(feature_importance_lasso[label]-1)
    
    return rcr, concordance, feature_importance_lasso

In [62]:
import matplotlib.pyplot as plt
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = 14

def get_survival_graph(rsf, rcr, X, lasso_X, y, label, filename):
    pred_surv_rsf = rsf.predict_survival_function(X)    
    pred_surv_rcr = rcr.predict_survival_function(lasso_X)
    
    # display survival plot
    plt.suptitle(label)
    plt.plot(np.mean([person for person in pred_surv_rsf], axis=0), label='RF')
    plt.plot(np.mean([person.y for person in pred_surv_rcr], axis=0), label='Lasso')
    labels, temp = get_ground_truth(y)
    plt.plot(labels, temp, label='Ground Truth')
    plt.legend()
    plt.xlim(0, 365)
    plt.xticks(np.arange(0, 365, step=50))
    plt.yticks(np.arange(0, 1.1, step=0.1))
    plt.savefig(filename)
        
    plt.show()

In [63]:
# helper function for plotting out ground truth curves

def get_ground_truth(data):
    relapsed = data[data.Illicit_Cens == 1]
    counts = relapsed['Illicit_Days'].value_counts()
    counts = counts.to_dict()
    temp = [len(data)] * 365
    labels = list(range(365))
    for i in range(365):
        labels[i] += 1
    total = 0
    errors = []
    for i in range(365):
        try:
            temp[i] = temp[i] - counts[i+1] - total
            total = total + counts[i+1]
        except KeyError:
            errors.append(i)

    for ele in sorted(errors, reverse = False):
        if ele != 0:
            temp[ele] = temp[ele-1]
        else:
             temp[0] = len(data)
    temp = [x / len(data) for x in temp]
    return labels, temp

In [64]:
# helper functions for displaying table data
from IPython.display import display_html

# n is the number of columns to display data in
def display_side_by_side(series_obj, n):
    df = pd.DataFrame(series_obj)
    partition = int(round(len(df) / n))
    lower_bound = 0
    upper_bound = partition
    args = []
    for i in range(n):
        args.append(df[lower_bound:upper_bound])
        lower_bound += partition
        upper_bound += partition
    helper(args)

def helper(args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

Survival Analysis by Severity

In [65]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('../data/data_superset.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ID,State,City,agyaddr,xobsyr_0,Illicit_Days,Illicit_Cens,female,nonwhite,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,SUDSy_0_cd,homeless_0_cd,ncar_cd,TRIg_0_cd,PYS9Sxg_cd,primsev_other,primsev_alcohol,primsev_amphetamines,primsev_cocaine,primsev_marijuana,primsev_opioids,B2a_0g,Address,lat,lng,state_name,county_FIPS,block_FIPS,murder_numg,%_dropoutg,%_povertyg,%_public_assistanceg,%_unemployedg,closest,gran,point,pop_deng
0,0,1,6,11520,TX,Laredo,"2386 E. Saunders, Suite 2",2007,78,1,0,1,0,0,2,0,1,1,0,0,2,1,2,1,2,3,0,1,1,0,0,0,0,0,1,0,1,"2386 E. Saunders, Suite 2 , L...",27.530608,-99.472335,Texas,48479.0,4.847900e+14,0,0.0,1.0,0.0,0.0,NaN,0.0,"('27.530608', '-99.472335')",0.0
1,1,2,7,19646,FL,Jacksonville,2671 Huffman Blvd,2011,365,0,0,1,0,0,0,0,2,1,1,0,0,1,2,0,0,1,0,1,1,0,0,1,0,0,0,0,1,"2671 Huffman Blvd , J...",30.298713,-81.504950,Florida,12031.0,1.203101e+14,0,0.0,0.0,0.0,0.0,NaN,0.0,"('30.298712599999998', '-81.50495040000001')",0.0
2,2,3,8,11738,TX,San Antonio,7703 Floyd Curl Drive,2008,188,0,0,1,0,0,0,0,0,0,0,0,1,2,2,0,0,2,0,1,0,0,0,0,0,0,1,0,1,"7703 Floyd Curl Drive , S...",29.513085,-98.577729,Texas,48029.0,4.802918e+14,0,0.0,0.0,0.0,0.0,NaN,0.0,"('29.5130847', '-98.57772890000001')",0.0
3,3,7,13,13054,AZ,Tucson,P.O. Box 3308,2007,365,0,0,1,1,1,2,0,0,1,1,1,2,2,2,0,1,3,0,0,0,0,0,0,1,0,0,0,1,"P.O. Box 3308 , T...",32.225221,-110.909508,Arizona,4019.0,4.019002e+13,0,0.0,0.0,0.0,0.0,NaN,1.0,"('32.225221000000005', '-110.909508')",0.0
4,4,10,16,15206,MA,Cambridge,349 Broadway,2008,337,0,1,1,0,0,2,0,2,0,0,0,0,1,2,0,0,1,1,1,1,0,0,0,1,0,0,0,2,"349 Broadway , C...",42.370383,-71.103350,Massachusetts,25017.0,2.501735e+14,0,0.0,0.0,0.0,0.0,NaN,0.0,"('42.3703833', '-71.1033498')",0.0


In [66]:
# drop unnecessary columns
cols_to_drop = ['Address','lat','lng','xobsyr_0','Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1',
                'ID','State','City','agyaddr','state_name','gran','county_FIPS','block_FIPS',
                'point','closest','%_public_assistanceg','%_dropoutg','%_unemployedg']

df.drop(columns=cols_to_drop, inplace=True)
df.dropna(inplace=True) # drops any remaining rows with null values

# uncomment to get CONTROL statistics
#cols_to_drop = ['pop_deng','%_dropoutg','%_unemployedg','%_public_assistanceg','%_povertyg','murder_numg']
#df.drop(columns=cols_to_drop, inplace=True)

df = df.astype(int)
df = df.sample(frac=1).reset_index(drop=True) # shuffle rows
df.shape

(10683, 32)

In [67]:
df["SUDSy_0_cd"].value_counts()

3    4354
1    3365
2    2964
Name: SUDSy_0_cd, dtype: int64

In [68]:
df.to_csv('../data/data_final.csv')
df.head()

,Illicit_Days,Illicit_Cens,female,nonwhite,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,SUDSy_0_cd,homeless_0_cd,ncar_cd,TRIg_0_cd,PYS9Sxg_cd,primsev_other,primsev_alcohol,primsev_amphetamines,primsev_cocaine,primsev_marijuana,primsev_opioids,B2a_0g,murder_numg,%_povertyg,pop_deng
0,352,0,0,1,0,0,2,2,2,0,2,1,1,1,2,0,1,2,1,1,1,0,0,0,0,0,1,0,2,0,1,0
1,72,1,1,1,0,0,2,0,1,2,1,0,0,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0
2,184,0,0,1,0,1,1,0,0,0,0,0,2,1,2,0,1,2,0,1,0,0,0,0,0,0,1,0,1,1,0,0
3,100,1,1,0,0,0,2,0,1,2,1,1,0,1,2,1,2,2,0,0,1,0,0,1,0,0,0,0,2,0,0,0
4,240,0,0,1,0,0,1,0,1,1,1,0,0,1,1,0,0,2,0,0,1,0,0,0,0,0,1,0,1,0,0,0


Full Population Analysis

In [69]:
from sksurv.util import Surv

predictor_var = 'Illicit_Days'
censoring_var = 'Illicit_Cens'
rf_threshold = 0.2

X = df.copy()
Y = X[[censoring_var, predictor_var]]
X.drop(columns=[censoring_var, predictor_var], inplace=True)
y = Surv.from_arrays(Y[censoring_var], Y[predictor_var]) # structured array to ensure correct censoring

print(X.shape, y.shape)

(10683, 30) (10683,)


In [ ]:
%%time
full_rsf, full_rsf_score, full_rsf_feature_importance = random_forest(X, y, 'ALL')
full_rf_features = full_rsf_feature_importance[full_rsf_feature_importance.ALL >= rf_threshold]['Feature'].tolist()

RF score: 0.6572859682606297


In [ ]:
%%time
full_lasso_X = get_lasso_features(X[full_rf_features])
full_rcr, full_rcr_score, full_rcr_feature_importance = lasso_regression(full_lasso_X, y, 'ALL')

In [ ]:
get_survival_graph(full_rsf, full_rcr, X, full_lasso_X, Y, 'Survival: All Severity Levels','../graphs/survival_all.png')

Subclinical Severity Analysis

In [ ]:
X = df[df.SUDSy_0_cd == 1]
Y = X[[censoring_var, predictor_var]]
X.drop(columns=[censoring_var, predictor_var, 'SUDSy_0_cd'], inplace=True)
 
y = Surv.from_arrays(Y[censoring_var], Y[predictor_var]) # structured array to ensure correct censoring

print(X.shape, y.shape)

In [ ]:
%%time
sub_rsf, sub_rsf_score, sub_rsf_feature_importance = random_forest(X, y, 'SUB')
sub_rf_features = sub_rsf_feature_importance[sub_rsf_feature_importance.SUB >= rf_threshold]['Feature'].tolist()

In [ ]:
%%time
sub_lasso_X = get_lasso_features(X[sub_rf_features])
sub_rcr, sub_rcr_score, sub_rcr_feature_importance = lasso_regression(sub_lasso_X, y, 'SUB')

In [ ]:
get_survival_graph(sub_rsf, sub_rcr, X, sub_lasso_X, Y, 'Survival: Subclinical Severity','../graphs/survival_sub.png')

Mild Severity Analysis

In [ ]:
X = df[df.SUDSy_0_cd == 2]
Y = X[[censoring_var, predictor_var]]
X.drop(columns=[censoring_var, predictor_var, 'SUDSy_0_cd'], inplace=True)

y = Surv.from_arrays(Y[censoring_var], Y[predictor_var]) # structured array to ensure correct censoring

print(X.shape, y.shape)

In [ ]:
%%time
mild_rsf, mild_rsf_score, mild_rsf_feature_importance = random_forest(X, y, 'MILD')
mild_rf_features = mild_rsf_feature_importance[mild_rsf_feature_importance.MILD >= rf_threshold]['Feature'].tolist()

In [ ]:
%%time
mild_lasso_X = get_lasso_features(X[mild_rf_features])
mild_rcr, mild_rcr_score, mild_rcr_feature_importance = lasso_regression(mild_lasso_X, y, 'MILD')

In [ ]:
get_survival_graph(mild_rsf, mild_rcr, X, mild_lasso_X, Y, 'Survival: Mild Severity','../graphs/survival_mild.png')

Severe Severity Analysis

In [ ]:
X = df[df.SUDSy_0_cd == 3]
Y = X[[censoring_var, predictor_var]]
X.drop(columns=[censoring_var, predictor_var, 'SUDSy_0_cd'], inplace=True)

y = Surv.from_arrays(Y[censoring_var], Y[predictor_var]) # structured array to ensure correct censoring

print(X.shape, y.shape)

In [ ]:
%%time
severe_rsf, severe_rsf_score, severe_rsf_feature_importance = random_forest(X, y, 'SEVERE')
severe_rf_features = severe_rsf_feature_importance[severe_rsf_feature_importance.SEVERE >= rf_threshold]['Feature'].tolist()

In [ ]:
severe_rsf_feature_importance[severe_rsf_feature_importance.SEVERE != 0]

In [ ]:
%%time
severe_lasso_X = get_lasso_features(X[severe_rf_features])
severe_rcr, severe_rcr_score, severe_rcr_feature_importance = lasso_regression(severe_lasso_X, y, 'SEVERE')

In [ ]:
get_survival_graph(severe_rsf, severe_rcr, X, severe_lasso_X, Y, 'Survival: Severe Severity','../graphs/survival_severe.png')

Performance Statistics

In [ ]:
# concordance index
scores = {'MODEL': ['Random Forest','Lasso','Dataset Size'], 
          'SUB': [sub_rsf_score,sub_rcr_score,int(sub_lasso_X.shape[0])],
          'MILD': [mild_rsf_score,mild_rcr_score,int(mild_lasso_X.shape[0])],
          'SEVERE': [severe_rsf_score,severe_rcr_score,int(severe_lasso_X.shape[0])],
          'FULL': [full_rsf_score,full_rcr_score,int(full_lasso_X.shape[0])]
         }
overall_concordance = pd.DataFrame(data=scores).round(4)
overall_concordance

Feature Importance - Random Forest

In [ ]:
overall_feature_importance_rf = pd.merge(sub_rsf_feature_importance, mild_rsf_feature_importance, on='Feature', how='outer')
overall_feature_importance_rf = pd.merge(overall_feature_importance_rf, severe_rsf_feature_importance, on='Feature', how='outer')
overall_feature_importance_rf.fillna(0, inplace=True)
display_side_by_side(overall_feature_importance_rf, 4)

In [ ]:
sub_feature_importance_rf = sub_rsf_feature_importance.nlargest(10,['SUB'])
mild_feature_importance_rf = mild_rsf_feature_importance.nlargest(10,['MILD'])
severe_feature_importance_rf = severe_rsf_feature_importance.nlargest(10,['SEVERE'])

top10_feature_importance_rf = pd.merge(sub_feature_importance_rf, mild_feature_importance_rf, on='Feature', how='outer')
top10_feature_importance_rf = pd.merge(top10_feature_importance_rf, severe_feature_importance_rf, on='Feature', how='outer')
top10_feature_importance_rf.fillna(0, inplace=True)
display_side_by_side(top10_feature_importance_rf, 4)

In [ ]:
# feature importance for rf across all ages
feature_importance = pd.DataFrame({'SUB': top10_feature_importance_rf['SUB'].tolist(),
                   'MILD': top10_feature_importance_rf['MILD'].tolist(),
                   'SEVERE': top10_feature_importance_rf['SEVERE'].tolist()},
                  index=top10_feature_importance_rf['Feature'].tolist())
# John asked to sort this graph by MILD
feature_importance.sort_values(by=['SUB','MILD','SEVERE'], ascending=False, inplace=True)
ax = feature_importance.plot.barh(rot=50, figsize=(12, 12))
ax.grid()
ax.set_title('Feature Importance: RF')
fig = ax.get_figure()
    
fig.savefig('../graphs/feature_importance.png', bbox_inches='tight')

In [ ]:
# feature importance for rf across all ages
feature_importance_sub = pd.DataFrame({'SUB': top10_feature_importance_rf['SUB'].tolist()},
                  index=top10_feature_importance_rf['Feature'].tolist())
feature_importance_sub = feature_importance_sub[feature_importance_sub.SUB != 0]
# John asked to sort this graph by MILD
feature_importance_sub.sort_values(by=['SUB'], ascending=False, inplace=True)
ax = feature_importance_sub.plot.barh(rot=50, figsize=(12, 12), color='blue')
ax.grid()
ax.set_title('Feature Importance: RF')
fig = ax.get_figure()
    
fig.savefig('../graphs/feature_importance_sub.png', bbox_inches='tight')

In [ ]:
# feature importance for rf across all ages
feature_importance_mild = pd.DataFrame({'MILD': top10_feature_importance_rf['MILD'].tolist()},
                  index=top10_feature_importance_rf['Feature'].tolist())
feature_importance_mild = feature_importance_mild[feature_importance_mild.MILD != 0]
# John asked to sort this graph by MILD
feature_importance_mild.sort_values(by=['MILD'], ascending=False, inplace=True)
ax = feature_importance_mild.plot.barh(rot=50, figsize=(12, 12), color='orange')
ax.grid()
ax.set_title('Feature Importance: RF')
fig = ax.get_figure()
    
fig.savefig('../graphs/feature_importance_mild.png', bbox_inches='tight')

In [ ]:
# feature importance for rf across all ages
feature_importance_severe = pd.DataFrame({'SEVERE': top10_feature_importance_rf['SEVERE'].tolist()},
                  index=top10_feature_importance_rf['Feature'].tolist())
feature_importance_severe = feature_importance_severe[feature_importance_severe.SEVERE != 0]
# John asked to sort this graph by MILD
feature_importance_severe.sort_values(by=['SEVERE'], ascending=False, inplace=True)
ax = feature_importance_severe.plot.barh(rot=50, figsize=(12, 12), color='green')
ax.grid()
ax.set_title('Feature Importance: RF')
fig = ax.get_figure()
    
fig.savefig('../graphs/feature_importance_severe.png', bbox_inches='tight')

Feature Importance - Lasso

In [ ]:
overall_feature_importance_lasso = pd.merge(sub_rcr_feature_importance, \
                                            mild_rcr_feature_importance, on='Feature', how='outer')
overall_feature_importance_lasso = pd.merge(overall_feature_importance_lasso, \
                                            severe_rcr_feature_importance, on='Feature', how='outer')
overall_feature_importance_lasso.fillna(0, inplace=True)
display_side_by_side(overall_feature_importance_lasso, 2)

In [ ]:
sub_feature_importance_lasso = sub_rcr_feature_importance.nlargest(10,['SUB_adjusted'])
mild_feature_importance_lasso = mild_rcr_feature_importance.nlargest(10,['MILD_adjusted'])
severe_feature_importance_lasso = severe_rcr_feature_importance.nlargest(10,['SEVERE_adjusted'])

top10_feature_importance_lasso = pd.merge(sub_feature_importance_lasso, \
                                            mild_feature_importance_lasso, on='Feature', how='outer')
top10_feature_importance_lasso = pd.merge(top10_feature_importance_lasso, \
                                            severe_feature_importance_lasso, on='Feature', how='outer')
top10_feature_importance_lasso.fillna(0, inplace=True)
display_side_by_side(top10_feature_importance_lasso, 2)

In [ ]:
top10_feature_importance_lasso['SUB_minus1'] = top10_feature_importance_lasso['SUB'] - 1
top10_feature_importance_lasso['MILD_minus1'] = top10_feature_importance_lasso['MILD'] - 1
top10_feature_importance_lasso['SEVERE_minus1'] = top10_feature_importance_lasso['SEVERE'] - 1

haz_df = pd.DataFrame({'SUB': top10_feature_importance_lasso['SUB_minus1'].tolist(),
                   'MILD': top10_feature_importance_lasso['MILD_minus1'].tolist(),
                   'SEVERE': top10_feature_importance_lasso['SEVERE_minus1'].tolist()},
                  index=top10_feature_importance_lasso['Feature'].tolist())
haz_df = haz_df.replace(-1, 0)
haz_df.sort_values(by=['SUB','MILD','SEVERE'], ascending=False, inplace=True)
ax = haz_df.plot.barh(rot=50, figsize=(12, 12))
ax.grid()
ax.set_title('Hazards From Lasso Model')
fig = ax.get_figure()

fig.savefig('../graphs/hazards_lasso.png', bbox_inches='tight')

In [ ]:
haz_sub = pd.DataFrame({'SUB': top10_feature_importance_lasso['SUB_minus1'].tolist()},
                  index=top10_feature_importance_lasso['Feature'].tolist())
haz_sub = haz_sub[haz_sub.SUB != -1]
haz_sub.sort_values(by=['SUB'], ascending=False, inplace=True)

ax = haz_sub.plot.barh(rot=50, figsize=(12, 12), color='blue')
ax.grid()
ax.set_title('Hazards From Lasso Model')
fig = ax.get_figure()

fig.savefig('../graphs/hazards_lasso_sub.png', bbox_inches='tight')

In [ ]:
haz_mild = pd.DataFrame({'MILD': top10_feature_importance_lasso['MILD_minus1'].tolist()},
                  index=top10_feature_importance_lasso['Feature'].tolist())
haz_mild = haz_mild[haz_mild.MILD != -1]
haz_mild.sort_values(by=['MILD'], ascending=False, inplace=True)

ax = haz_mild.plot.barh(rot=50, figsize=(12, 12), color='orange')
ax.grid()
ax.set_title('Hazards From Lasso Model')
fig = ax.get_figure()

fig.savefig('../graphs/hazards_lasso_mild.png', bbox_inches='tight')

In [ ]:
haz_severe = pd.DataFrame({'SEVERE': top10_feature_importance_lasso['SEVERE_minus1'].tolist()},
                  index=top10_feature_importance_lasso['Feature'].tolist())
haz_severe = haz_severe[haz_severe.SEVERE != -1]
haz_severe.sort_values(by=['SEVERE'], ascending=False, inplace=True)

ax = haz_severe.plot.barh(rot=50, figsize=(12, 12), color='green')
ax.grid()
ax.set_title('Hazards From Lasso Model')
fig = ax.get_figure()

fig.savefig('../graphs/hazards_lasso_severe.png', bbox_inches='tight')

Saving Feature Importance Spreadsheet...

In [ ]:
def find_lasso_winner(feat, lasso_0, lasso_2):
    if lasso_0 == 0 and lasso_2 == 0: # rf deemed unimportant
        return pd.Series((0, ''))
    bin_features = ['female','nonwhite','unemplmt_cd','primsev_alcohol','primsev_amphetamines','homeless_0_cd',
                    'primsev_cocaine','primsev_marijuana','primsev_opioids','primsev_other']
    if feat in bin_features:
        return pd.Series((lasso_0, 2))
    elif lasso_0 == 0:
        return pd.Series((lasso_2, 2))
    elif lasso_2 == 0:
        return pd.Series((lasso_0, 0))
    elif abs(lasso_0-1) > abs(lasso_2-1):
        return pd.Series((lasso_0, '0'))
    else:
        return pd.Series((lasso_2, '2'))

In [ ]:
sub_cmn_feat = sub_rsf_feature_importance.copy()
sub_cmn_feat['Lasso_0'] = 0.0
sub_cmn_feat['Lasso_2'] = 0.0
sub_lasso_feat = sub_rcr_feature_importance.copy()
for i,row_rf in tqdm(sub_cmn_feat.iterrows(), total=sub_cmn_feat.shape[0]):
    rf_feat = row_rf['Feature']
    for j,row_lasso in sub_lasso_feat.iterrows():
        lasso_feat = row_lasso['Feature']
        if lasso_feat.startswith(rf_feat):
            if lasso_feat[-1] == '2':
                sub_cmn_feat.set_value(i, 'Lasso_2', row_lasso['SUB'])
            else:
                sub_cmn_feat.set_value(i, 'Lasso_0', row_lasso['SUB'])
sub_cmn_feat.sort_values(by='SUB', inplace=True, ascending=False)
sub_cmn_feat[['lasso_max','lasso_max_level']] = sub_cmn_feat.apply(lambda x: find_lasso_winner(x['Feature'], x['Lasso_0'], x['Lasso_2']), axis=1)
sub_cmn_feat

In [ ]:
mild_cmn_feat = mild_rsf_feature_importance.copy()
mild_cmn_feat['Lasso_0'] = 0.0
mild_cmn_feat['Lasso_2'] = 0.0
mild_lasso_feat = mild_rcr_feature_importance.copy()
for i,row_rf in tqdm(mild_cmn_feat.iterrows(), total=mild_cmn_feat.shape[0]):
    rf_feat = row_rf['Feature']
    for j,row_lasso in mild_lasso_feat.iterrows():
        lasso_feat = row_lasso['Feature']
        if lasso_feat.startswith(rf_feat):
            if lasso_feat[-1] == '2':
                mild_cmn_feat.set_value(i, 'Lasso_2', row_lasso['MILD'])
            else:
                mild_cmn_feat.set_value(i, 'Lasso_0', row_lasso['MILD'])
mild_cmn_feat.sort_values(by='MILD', inplace=True, ascending=False)
mild_cmn_feat[['lasso_max','lasso_max_level']] = mild_cmn_feat.apply(lambda x: find_lasso_winner(x['Feature'], x['Lasso_0'], x['Lasso_2']), axis=1)
mild_cmn_feat

In [ ]:
severe_cmn_feat = severe_rsf_feature_importance.copy()
severe_cmn_feat['Lasso_0'] = 0.0
severe_cmn_feat['Lasso_2'] = 0.0
severe_lasso_feat = severe_rcr_feature_importance.copy()
for i,row_rf in tqdm(severe_cmn_feat.iterrows(), total=severe_cmn_feat.shape[0]):
    rf_feat = row_rf['Feature']
    for j,row_lasso in severe_lasso_feat.iterrows():
        lasso_feat = row_lasso['Feature']
        if lasso_feat.startswith(rf_feat):
            if lasso_feat[-1] == '2':
                severe_cmn_feat.set_value(i, 'Lasso_2', row_lasso['SEVERE'])
            else:
                severe_cmn_feat.set_value(i, 'Lasso_0', row_lasso['SEVERE'])
severe_cmn_feat.sort_values(by='SEVERE', inplace=True, ascending=False)
severe_cmn_feat[['lasso_max','lasso_max_level']] = severe_cmn_feat.apply(lambda x: find_lasso_winner(x['Feature'], x['Lasso_0'], x['Lasso_2']), axis=1)
severe_cmn_feat

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('../graphs/common_feature_analysis.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
sub_cmn_feat.to_excel(writer, sheet_name='Subclinical')
mild_cmn_feat.to_excel(writer, sheet_name='Mild')
severe_cmn_feat.to_excel(writer, sheet_name='Severe')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
%%bash
jupyter nbconvert --to html ./survival_analysis.ipynb
mv survival_analysis.html ../graphs/survival_analysis_2.html

In [ ]:
# print out total notebook execution time
total_seconds = int(time.time() - start_time)
hours = total_seconds // (60 * 60)
minutes = (total_seconds - hours*60) // 60
seconds = (total_seconds - hours*60) % 60
print("--- " + str(minutes) + " minutes " + str(seconds) + " seconds ---")